# quake 3 server connector



## channel




### channel message poller



#### the code

poll discord channel?



In [ ]:
var fs = require('fs')
var {request} = require('gaxios')
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var credentials
var tokenPath
if(fs.existsSync('./discord-bot.json')) {
    tokenPath = path.resolve('./discord-bot.txt')
} else {
    tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}

async function authorizeDiscord() {
    var {client_id, client_secret} = JSON.parse(fs.readFileSync(credentials).toString('utf-8'))
    if(fs.existsSync(tokenPath))
        var {access_token, expires} = JSON.parse(fs.readFileSync(tokenPath).toString('utf-8'))
    if(access_token && Date.now() < expires) return access_token
    var data = `grant_type=client_credentials&scope=identify%20connections%20bot%20guilds%20messages.read%20rpc`
    var result = await request({
        headers: {
            'Content-Type': 'application/x-www-form-urlencoded',
            Authorization: 'Basic ' + Buffer.from(`${client_id}:${client_secret}`).toString('base64')
        },
        method: 'POST',
        url: `https://discord.com/api/v6/oauth2/token`,
        data
    })
    fs.writeFileSync(tokenPath, JSON.stringify(Object.assign({}, result.data, {
        expires: (result.data.expires_in * 1000) + Date.now()}), null, 2))
    return result.data.access_token
}

async function userGuilds(userId = '@me') {
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `https://discord.com/api/v6/users/${userId}/guilds`
    })
    return result.data
}

async function guildChannels(guildId = '393252386426191873') {
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `https://discord.com/api/v6/guilds/${guildId}/channels`
    })
    return result.data
}

async function channelMessages(channelId = '393252386426191875') {
    var params = {
        after: (BigInt(Date.now() - 1420070400000 - 1000 * 65) << 22n).toString()
    };
    var token = fs.readFileSync(tokenPath).toString('utf-8').trim()
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `https://discord.com/api/v6/channels/${channelId}/messages`,
        params
    })
    return result.data
}

module.exports = userGuilds
